In [22]:
import numpy as np
import pandas as pd
import datetime as dt
import xarray as xr
from urllib.error import URLError

import sys
sys.path.append("/home/elilouis/sublimationofsnow/")
import sosutils

import altair as alt
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
alt.renderers.enable("jupyterlab")

RendererRegistry.enable('jupyterlab')

In [23]:
sos_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc'
DATE_FORMAT_STR = '%Y%m%d'
start_date = '20230315'; 
end_date = '20230315'; 

datelist = pd.date_range(
    dt.datetime.strptime(start_date, DATE_FORMAT_STR),
    dt.datetime.strptime(end_date, DATE_FORMAT_STR),
    freq='d'
).strftime(DATE_FORMAT_STR).tolist()

VARIABLE_NAMES = [
    # Snow-level temperature arrays (tower D)
    'Tsnow_0_4m_d', 'Tsnow_0_5m_d', 'Tsnow_0_6m_d', 'Tsnow_0_7m_d', 'Tsnow_0_8m_d', 'Tsnow_0_9m_d', 'Tsnow_1_0m_d', 'Tsnow_1_1m_d', 'Tsnow_1_2m_d', 'Tsnow_1_3m_d', 'Tsnow_1_4m_d', 'Tsnow_1_5m_d',

    'Tsnow_0_4m_uw', 'Tsnow_0_5m_uw', 'Tsnow_0_6m_uw', 'Tsnow_0_7m_uw', 'Tsnow_0_8m_uw', 'Tsnow_0_9m_uw', 'Tsnow_1_0m_uw', 'Tsnow_1_1m_uw', 'Tsnow_1_2m_uw', 'Tsnow_1_3m_uw', 'Tsnow_1_4m_uw', 'Tsnow_1_5m_uw',
]

In [24]:
# We make sure that we aren't accessing variables that don't exist in the datasets
# This is necessary because some daily NetCDF files don't have all the expected variables
# (for example because an instrument was down). In that case, we want to add that variable
# to the dataset, filled with nans, which sosutils.merge_datasets_with_different_variables
# handles for us
datasets = []
datasets_safe = []
for date in datelist:
    try:
        ds = xr.open_dataset(sosutils.download_sos_data_day(date, sos_download_dir, cache=True))
    # Some dates are missing
    except URLError:
        print(f"failed on {date}, skipping")
    ds_new = ds[set(ds.data_vars).intersection(VARIABLE_NAMES)]
    datasets.append(ds_new)
    datasets_safe.append(ds_new)
sos_ds = sosutils.merge_datasets_with_different_variables(datasets, dim='time')

Caching...skipping download for 20230315


In [25]:
df = sosutils.get_tidy_dataset(
    sos_ds,
    [
        'Tsnow_0_4m_d',
        'Tsnow_0_5m_d',
        'Tsnow_0_6m_d',
        'Tsnow_0_7m_d',
        'Tsnow_0_8m_d',
        'Tsnow_0_9m_d',
        'Tsnow_1_0m_d',
        'Tsnow_1_1m_d',
        'Tsnow_1_2m_d',
        'Tsnow_1_3m_d',
        'Tsnow_1_4m_d',
        'Tsnow_1_5m_d',

        'Tsnow_0_4m_uw',
        'Tsnow_0_5m_uw',
        'Tsnow_0_6m_uw',
        'Tsnow_0_7m_uw',
        'Tsnow_0_8m_uw',
        'Tsnow_0_9m_uw',
        'Tsnow_1_0m_uw',
        'Tsnow_1_1m_uw',
        'Tsnow_1_2m_uw',
        'Tsnow_1_3m_uw',
        'Tsnow_1_4m_uw',
        'Tsnow_1_5m_uw',
    ]
)

In [26]:
alt.Chart(df).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
    alt.Color("height:O").scale(scheme='viridis'),
    alt.Facet("tower:N", columns=1)
).properties(width=900, height = 250)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


The 2D heat equation is
$$
\frac{\partial}{\partial t}  T(z,t) = \alpha \frac{\partial^2}{\partial z^2} T(z,t)
$$

We discretize this equation with

$$z_i = i \Delta z$$
$$t_j = j \Delta t$$
$$T(z,t) = T_i^j$$

Using a simple forward differencing scheme to estimate derivatives, we can discretize the above equation and solve for $k^2$.

$$ \alpha =  \frac{T_i^{j+1} - T_i^{j}}{\Delta t} \Big( \frac{\Delta z^2}{T_{i+1}^{j} - 2 T_{i}^{j} + T_{i-1}^{j}} \Big)$$

Note that this scheme is numerically stable when 

$$ \Delta t \leq \frac{\Delta x^2}{4 k^2}$$

The solution for $k^2$ allows us to solve for k for every data point in the (z,t) space that isn't on the edge of the (z,t) grid.

In our dataset, $\Delta t$ is 300 seconds and $\Delta x$ is 0.1 meters. We now transform our dataset to be in a 2-D array.

In [27]:
Tsnow_d = np.array([
    sos_ds['Tsnow_0_4m_d'].values,
    sos_ds['Tsnow_0_5m_d'].values,
    sos_ds['Tsnow_0_6m_d'].values,
    sos_ds['Tsnow_0_7m_d'].values,
    sos_ds['Tsnow_0_8m_d'].values,
    sos_ds['Tsnow_0_9m_d'].values,
    sos_ds['Tsnow_1_0m_d'].values,
    sos_ds['Tsnow_1_1m_d'].values,
    sos_ds['Tsnow_1_2m_d'].values,
    sos_ds['Tsnow_1_3m_d'].values,
    sos_ds['Tsnow_1_4m_d'].values,
    sos_ds['Tsnow_1_5m_d'].values
])

Tsnow_uw = np.array([
    sos_ds['Tsnow_0_4m_uw'].values,
    sos_ds['Tsnow_0_5m_uw'].values,
    sos_ds['Tsnow_0_6m_uw'].values,
    sos_ds['Tsnow_0_7m_uw'].values,
    sos_ds['Tsnow_0_8m_uw'].values,
    sos_ds['Tsnow_0_9m_uw'].values,
    sos_ds['Tsnow_1_0m_uw'].values,
    sos_ds['Tsnow_1_1m_uw'].values,
    sos_ds['Tsnow_1_2m_uw'].values,
    sos_ds['Tsnow_1_3m_uw'].values,
    sos_ds['Tsnow_1_4m_uw'].values,
    sos_ds['Tsnow_1_5m_uw'].values
])

This grid allows us to access our data with $i$ and $j$ indices like so:

In [28]:
i = 0; j = 0
print(Tsnow_d[i][j])
i = 0; j = 1
print(Tsnow_d[i][j])
i = 1; j = 0
print(Tsnow_d[i][j])
i = 1; j = 1
print(Tsnow_d[i][j])

-0.043280188
-0.0450607
-1.2413691
-1.2393041


In [29]:
Tsnow_d.shape

(12, 288)

Now we can step through our space and time grid. We have 12 spatial measurements each with 288 time steps.

$i \quad \epsilon \quad [0, 11]$

$j \quad \epsilon \quad [0, 287]$

where $i$ corresponds to measurements between 0.4 and 1.5 meters, at 0.1 meter increments.

Using our equation for $k^2$ above, repeated here

$$ \alpha =  \frac{T_i^{j+1} - T_i^{j}}{\Delta t} \Big( \frac{\Delta z^2}{T_{i+1}^{j} - 2 T_{i}^{j} + T_{i-1}^{j}} \Big)$$


We see that we can only calculate k^2 with the following subset of $i$ and $j$

$i \quad \epsilon \quad (0, 11)$

$j \quad \epsilon \quad [0, 287)$

Let's step through this data and calculate k^2! But first let's convert to Kelvin :).

In [30]:
Tsnow_d = Tsnow_d + 273.15
Tsnow_uw = Tsnow_uw + 273.15

In [31]:
delta_z = 0.1
delta_t = 300

alpha_d = np.zeros_like(Tsnow_d)
alpha_uw = np.zeros_like(Tsnow_uw)
for i in range(1,11):
    for j in range(0, 287):
        alpha_d[i][j] = (
            (Tsnow_d[i][j+1] - Tsnow_d[i][j])/delta_t
        )*(
            delta_z**2 / ( Tsnow_d[i+1][j] - 2*Tsnow_d[i][j] + Tsnow_d[i-1][j])
        )

        alpha_uw[i][j] = (
            (Tsnow_uw[i][j+1] - Tsnow_uw[i][j])/delta_t
        )*(
            delta_z**2 / ( Tsnow_uw[i+1][j] - 2*Tsnow_uw[i][j] + Tsnow_uw[i-1][j])
        )

In [32]:
np.nanmean(alpha_d), np.nanmedian(alpha_d), np.max(alpha_d), np.min(alpha_d)

(7.126021e-08, 2.7379741e-08, 0.0007872549, -0.0008510101)

In [33]:
np.nanmean(alpha_uw), np.nanmedian(alpha_uw), np.max(alpha_uw), np.min(alpha_uw)

(-1.0018993e-07, 0.0, 7.962963e-05, -0.00017700618)

In [34]:
src = pd.DataFrame({
    'thermal diffusivity (tower d)': alpha_d.flatten(),
    'thermal diffusivity (tower uw)': alpha_uw.flatten()
})
alt.Chart(src).mark_bar().transform_fold(
    ['thermal diffusivity (tower d)', 'thermal diffusivity (tower uw)']
).encode(
    alt.X("value:Q").title("α"),
    alt.Y("count():Q"),
    alt.Facet("key:N").title(None)
).properties(height=150)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
